In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.listdir('/content/drive/My Drive'))


['Classroom', 'zoom_0 (1).mp4', 'zoom_0.mp4', 'ömer writing 1 (1).docx', 'ömer weekly translation exercise modals 1,2.docx', 'ömer adjectives adverbs örnek cümleler.docx', 'ömer weekly translation passive exercise 1-2.docx', 'ömer weekly translation exercises relative clause.docx', 'ömer weekly translation exercises noun clause.docx', 'ömer weekly translation exercises if clauses.docx', 'ömer writing 1.docx', 'ömer writing 2.docx', 'ömer writing 3.docx', 'ömer writing 4.gdoc', 'ömer writing 5.docx', 'Kardeslere Ait Bilgiler.docx', 'Screenshot 2021-09-09 113901.png', 'yok-ogrenci-belgesi-sorgulama (1).pdf', 'nvi-nufus-aile kayit-ornegi-belgesi-sorgulama.pdf', 'adli-sicil-kaydi (1).pdf', 'Başlıksız e-tablo (1).gsheet', 'Başlıksız e-tablo.gsheet', 'p_transcript (1).pdf', 'uc.png', 'WhatsApp Image 2022-05-17 at 18.06.31.jpeg', 'fizik 2', '75448869_132391081083378_1851550492647236146_n.jpg adlı dosyanın kopyası (1)', '75448869_132391081083378_1851550492647236146_n.jpg adlı d

In [ ]:
print(os.listdir('/content/drive/My Drive/Colab_Data'))


['train_feats.npy', 'train_labels.csv', 'valtest_feats.npy', 'submission.csv', 'submission_Deep_Dense_Dropout_BN_L2_f1_0.98863.csv', 'submission_Improved_Deep_Dense_L2_BN_Ensemble_f1_0.98727.csv']


In [ ]:
import numpy as np
import pandas as pd

# Dosya yollarını belirtin
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# Dosyaları yükleyin
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

print("Dosyalar başarıyla yüklendi!")


Dosyalar başarıyla yüklendi!


In [ ]:
print("train_feats keys:", train_feats.keys())
print("train_labels head:", train_labels.head())
print("valtest_feats keys:", valtest_feats.keys())


train_feats keys: dict_keys(['idx', 'resnet_feature', 'vit_feature', 'clip_feature', 'dino_feature'])
train_labels head:    ID  label
0   0      1
1   1      3
2   2      8
3   3      7
4   4      9
valtest_feats keys: dict_keys(['idx', 'resnet_feature', 'vit_feature', 'clip_feature', 'dino_feature'])


In [ ]:
# Gerekli kütüphaneleri içe aktarın
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# --- 1. Google Drive Bağlantısı ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2. Dosya Yolları ---
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# --- 3. Veriyi Yükle ---
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

# Eğitim verisinden özellik vektörlerini ayır
resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])  # Özellik vektörlerini birleştir
y = train_labels['label'].values  # Etiketler

# Test verisinden özellik vektörlerini ayır
valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']

X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# --- 4. Veriyi Hazırlama ---
# Veriyi eğitim ve doğrulama olarak ayır
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Veriyi ölçeklendir
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# --- 5. Modeli Tanımlama ---
# Derin Sinir Ağı Modeli
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')  # 10 sınıf olduğu için çıkış katmanında 10 nöron
])

# Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# --- 6. Erken Durdurma Mekanizması ---
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --- 7. Modeli Eğitme ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping]
)

# --- 8. Doğrulama Performansı ---
# Doğrulama verisinde tahmin yap
y_val_pred = np.argmax(model.predict(X_val), axis=1)
macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Validation Macro F1 Score: {macro_f1}")

# --- 9. Test Verisi Tahmini ve Kaggle Formatı ---
test_predictions = np.argmax(model.predict(X_test), axis=1)

# Kaggle formatında submission dosyası hazırlama
submission = pd.DataFrame({
    'ID': range(len(test_predictions)),
    'Predicted': test_predictions
})

submission_path = '/content/drive/My Drive/Colab_Data/submission.csv'
submission.to_csv(submission_path, index=False)
print(f"Submission dosyası {submission_path} olarak kaydedildi.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9159 - loss: 0.2746 - val_accuracy: 0.9829 - val_loss: 0.0609
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9854 - loss: 0.0523 - val_accuracy: 0.9846 - val_loss: 0.0611
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9891 - loss: 0.0349 - val_accuracy: 0.9839 - val_loss: 0.0685
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9916 - loss: 0.0297 - val_accuracy: 0.9837 - val_loss: 0.0763
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9925 - loss: 0.0246 - val_accuracy: 0.9856 - val_loss: 0.0734
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9927 - loss: 0.0258 - val_accuracy: 0.9869 - val_loss: 0.0772
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Validation Macro F1 Score: 0.9829355345299977
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Submission dosyası /content/drive/My Drive/Colab_Data/submission.csv olarak kaydedildi.


In [ ]:
# Gerekli kütüphaneleri yükle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# --- 1. Google Drive Bağlantısı ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2. Dosya Yolları ---
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# --- 3. Veriyi Yükle ---
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

# Eğitim verisinden özellik vektörlerini ayır
resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])  # Özellik vektörlerini birleştir
y = train_labels['label'].values  # Etiketler

# Test verisinden özellik vektörlerini ayır
valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']

X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# --- 4. Veriyi Hazırlama ---
# Veriyi eğitim ve doğrulama olarak ayır
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Veriyi ölçeklendir
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# --- 5. Modeli Tanımlama ---
# Daha karmaşık model
model_name = "Deep_Dense_Dropout_BN_L2"
model = Sequential([
    Dense(1024, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(10, activation='softmax')  # 10 sınıf olduğu için çıkış katmanında 10 nöron
])

# Modeli derleme
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- 6. Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# --- 7. Modeli Eğitme ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[early_stopping, reduce_lr]
)

# --- 8. Doğrulama Performansı ---
# Doğrulama verisinde tahmin yap
y_val_pred = np.argmax(model.predict(X_val), axis=1)
macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Validation Macro F1 Score: {macro_f1}")

# --- 9. Test Verisi Tahmini ve Kaggle Formatı ---
test_predictions = np.argmax(model.predict(X_test), axis=1)

# Kaggle formatında submission dosyası hazırlama
submission_name = f'submission_{model_name}_f1_{macro_f1:.5f}.csv'
submission = pd.DataFrame({
    'ID': range(len(test_predictions)),
    'Predicted': test_predictions
})

submission_path = f'/content/drive/My Drive/Colab_Data/{submission_name}'
submission.to_csv(submission_path, index=False)
print(f"Submission dosyası {submission_path} olarak kaydedildi.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.8647 - loss: 3.0502 - val_accuracy: 0.9861 - val_loss: 1.8959 - learning_rate: 0.0010
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9837 - loss: 1.6595 - val_accuracy: 0.9843 - val_loss: 1.0480 - learning_rate: 0.0010
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9835 - loss: 0.9179 - val_accuracy: 0.9811 - val_loss: 0.6342 - learning_rate: 0.0010
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9848 - loss: 0.5555 - val_accuracy: 0.9814 - val_loss: 0.4284 - learning_rate: 0.0010
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9840 - loss: 0.3920 - val_accuracy: 0.9796 - val_loss: 0.3635 - learning_rate: 0.0010
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9846 - loss: 0.3322 - val_accuracy: 0.9793 - val_loss: 0.3196 - learning_rate: 0.0010
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9825 - loss: 

In [ ]:
# Gerekli kütüphaneleri yükle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# --- 1. Google Drive Bağlantısı ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2. Dosya Yolları ---
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# --- 3. Veriyi Yükle ---
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

# Eğitim verisinden özellik vektörlerini ayır
resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])  # Özellik vektörlerini birleştir
y = train_labels['label'].values  # Etiketler

# Test verisinden özellik vektörlerini ayır
valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']

X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# --- 4. Veriyi Hazırlama ---
# Veriyi eğitim ve doğrulama olarak ayır
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Veriyi ölçeklendir
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# --- 5. Modeli Tanımlama ---
# Daha karmaşık model
model_name = "Improved_Deep_Dense_L2_BN_Ensemble"
model = Sequential([
    Dense(2048, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 sınıf olduğu için çıkış katmanında 10 nöron
])

# Modeli derleme
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- 6. Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

# --- 7. Modeli Eğitme ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

# --- 8. Doğrulama Performansı ---
# Doğrulama verisinde tahmin yap
y_val_pred = np.argmax(model.predict(X_val), axis=1)
macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Validation Macro F1 Score: {macro_f1}")

# --- 9. Test Verisi Tahmini ve Kaggle Formatı ---
test_predictions = np.argmax(model.predict(X_test), axis=1)

# Kaggle formatında submission dosyası hazırlama
submission_name = f'submission_{model_name}_f1_{macro_f1:.5f}.csv'
submission = pd.DataFrame({
    'ID': range(len(test_predictions)),
    'Predicted': test_predictions
})

submission_path = f'/content/drive/My Drive/Colab_Data/{submission_name}'
submission.to_csv(submission_path, index=False)
print(f"Submission dosyası {submission_path} olarak kaydedildi.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.9212 - loss: 7.2990 - val_accuracy: 0.9746 - val_loss: 2.0165 - learning_rate: 0.0010
Epoch 2/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9689 - loss: 1.5440 - val_accuracy: 0.9759 - val_loss: 0.7394 - learning_rate: 0.0010
Epoch 3/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9654 - loss: 0.8286 - val_accuracy: 0.9468 - val_loss: 0.8192 - learning_rate: 0.0010
Epoch 4/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9644 - loss: 0.8088 - val_accuracy: 0.9728 - val_loss: 0.8422 - learning_rate: 0.0010
Epoch 5/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9681 - loss: 0.8418 - val_accuracy: 0.9777 - val_loss: 0.7438 - learning_rate: 0.0010
Epoch 6/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9640 - loss: 0.8078 - val_accuracy: 0.9679 - val_loss: 0.8256 - learning_rate: 0.0010
Epoch 7/100
426/438 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9645 - loss: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.decomposition import PCA
from tensorflow.keras.regularizers import l2

# --- 1. Veriyi Yükle ---
# Google Drive üzerinden dosyaları yüklediğiniz yolu kontrol edin
train_feats_path = '/content/drive/MyDrive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/MyDrive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/MyDrive/Colab_Data/valtest_feats.npy'

# Eğitim ve test özellik vektörlerini yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

# Özellikleri birleştir
resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

# Test setini ayarla
valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']

X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# --- 2. PCA ile Boyut Azaltma ---
pca = PCA(n_components=256)  # 256 bileşene düşür
X = pca.fit_transform(X)
X_test = pca.transform(X_test)

# --- 3. Eğitim ve Doğrulama Seti ---
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 4. Veriyi Ölçeklendir ---
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# --- 5. Model Tanımı ---
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(10, activation='softmax')  # Çok sınıflı sınıflandırma
])

# --- 6. Modeli Derle ---
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- 7. Callbacks ---
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1
)

# --- 8. Model Eğitimi ---
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # Daha yüksek bir epoch sayısı deneme yapılabilir
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

# --- 9. Test Seti Tahmini ---
test_predictions = model.predict(X_test)
predicted_classes = np.argmax(test_predictions, axis=1)

# --- 10. Submission Dosyasını Oluştur ---
submission = pd.DataFrame({
    'ID': range(len(predicted_classes)),
    'Predicted': predicted_classes
})

# Submission dosyasını kaydet
submission_file_path = '/content/drive/MyDrive/submission_optimized_pca.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission dosyası kaydedildi: {submission_file_path}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7010 - loss: 5.1349 - val_accuracy: 0.9829 - val_loss: 0.4899 - learning_rate: 0.0010
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9755 - loss: 0.4900 - val_accuracy: 0.9766 - val_loss: 0.4311 - learning_rate: 0.0010
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9715 - loss: 0.4488 - val_accuracy: 0.9761 - val_loss: 0.4140 - learning_rate: 0.0010
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9747 - loss: 0.4202 - val_accuracy: 0.9778 - val_loss: 0.3895 - learning_rate: 0.0010
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9727 - loss: 0.4105 - val_accuracy: 0.9781 - val_loss: 0.3559 - learning_rate: 0.0010
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9723 - loss: 0.3859 - val_accuracy: 0.9781 - val_loss: 0.3551 - learning_rate: 0.0010
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9743 - loss: 0.3719 - val_a

In [ ]:
# --- 1. Gerekli kütüphaneleri yükle ---
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# Google Drive bağlantısı
from google.colab import drive
drive.mount('/content/drive')

# Dosya yolları
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# Veriyi yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

# Özellikleri birleştir
resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']
X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# Model oluşturma fonksiyonu (fonksiyonel hale getirdim, böylece cross-validation'da tekrar kullanabiliriz)
def create_model(input_dim):
    model = Sequential([
        Dense(1024, activation='relu', input_dim=input_dim, kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),  # Daha düşük dropout ile başla
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Biraz daha küçük LR
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
best_models = []
for train_index, val_index in skf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Scale işlemi
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    model = create_model(X_train.shape[1])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=128,
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )

    y_val_pred = np.argmax(model.predict(X_val), axis=1)
    fold_f1 = f1_score(y_val, y_val_pred, average='macro')
    cv_scores.append(fold_f1)
    best_models.append(model)

print("CV F1 Skorları:", cv_scores)
print("Ortalama CV F1:", np.mean(cv_scores))

# Tüm veri üzerinde son modeli eğitme (opsiyonel)
# Burada en iyi modeli seçmek istersen ek işlem gerekebilir
final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(X)
final_model = create_model(X_scaled.shape[1])
final_model.fit(
    X_scaled, y,
    epochs=50,
    batch_size=128,
    verbose=1
)

# Test verisini tahminle
X_test_scaled = final_scaler.transform(X_test)
test_preds = np.argmax(final_model.predict(X_test_scaled), axis=1)

submission_name = f'submission_DeepCV.csv'
submission = pd.DataFrame({'ID': range(len(test_preds)), 'Predicted': test_preds})
submission_path = f'/content/drive/My Drive/Colab_Data/{submission_name}'
submission.to_csv(submission_path, index=False)
print(f"Submission dosyası {submission_path} olarak kaydedildi. 🤖")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 30: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 38: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 32: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 38: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 50: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


...


Epoch 19: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 25: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 31: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 39: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 48: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 30: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 37: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 48: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
CV F1 Skorları: [0.9887455974249901, 0.9872649483675915, 0.9886185840522733, 0.9868809973018043, 0.9889990584313125]
Ortalama CV F1: 0.9881018371155944


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.8862 - loss: 2.9540
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9866 - loss: 1.8731
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9873 - loss: 1.1709
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9891 - loss: 0.7138
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9888 - loss: 0.4561
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9881 - loss: 0.3128
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9876 - loss: 0.2483
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9872 - loss: 0.2143
Epoch 9/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9870 - loss: 0.2003
Epoch 10/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9882 - loss: 0.1939
Epoch 11/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9875 - loss: 0.1861
Epoch 12/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/st

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# Google Drive Bağlama
from google.colab import drive
drive.mount('/content/drive')

# Dosya Yolları
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# Verileri Yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# Mixup (isteğe bağlı veri artırma)
def mixup_data(X, y, alpha=0.2):
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X + (1 - lam) * X_shuffled
    # Basit "hard label" yaklaşımı
    return X_mix, y, lam, y_shuffled

# F1 Callback
class F1Callback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        # Doğrulama verisinde tahmin
        y_pred_proba = self.model.predict(self.X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(self.y_val, y_pred, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        # En iyi F1 skorunu güncelle
        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            # Dikkat: Keras son sürümlerde .weights.h5 uzantısı istiyor
            self.model.save_weights("best_f1_model.weights.h5")

# ExponentialDecay Planı
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.95,
    staircase=False
)

# Model Oluşturma Fonksiyonu
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.0005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='softmax'))

    # Adam optimizer'ı string yerine doğrudan veriyoruz:
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Stratified K-Fold Uygulama
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
val_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n===== FOLD {fold+1} =====")
    X_train_cv, X_val_cv = X[train_idx], X[val_idx]
    y_train_cv, y_val_cv = y[train_idx], y[val_idx]

    # Özellik ölçekleme
    scaler = StandardScaler()
    X_train_cv = scaler.fit_transform(X_train_cv)
    X_val_cv = scaler.transform(X_val_cv)

    # İsteğe bağlı Mixup
    X_train_mix, y_train_mix, _, _ = mixup_data(X_train_cv, y_train_cv, alpha=0.2)

    # Modeli oluştur
    model = create_model(X_train_cv.shape[1])

    # Callback'ler
    f1_cb = F1Callback(X_val_cv, y_val_cv)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

    # Eğit
    model.fit(
        X_train_mix, y_train_mix,
        validation_data=(X_val_cv, y_val_cv),
        epochs=30,
        batch_size=128,
        callbacks=[f1_cb, early_stopping, reduce_lr],
        verbose=1
    )

    # En iyi ağırlıkları geri yükle
    model.load_weights("best_f1_model.weights.h5")

    # Bu fold'un F1 skoru
    val_pred = model.predict(X_val_cv)
    y_val_pred = np.argmax(val_pred, axis=1)
    f1_cv = f1_score(y_val_cv, y_val_pred, average='macro')
    print(f"Fold {fold+1} Makro F1 => {f1_cv:.5f}")
    val_scores.append(f1_cv)

mean_f1 = np.mean(val_scores)
print("\nCV Makro F1 Skorları:", val_scores)
print(f"Ortalama Makro F1 => {mean_f1:.5f}")

# Tüm veriyle final model eğit
final_scaler = StandardScaler()
X_scaled = final_scaler.fit_transform(X)
final_model = create_model(X_scaled.shape[1])
final_model.fit(
    X_scaled, y,
    epochs=20,
    batch_size=128,
    verbose=1
)

# Test seti tahmin
X_test_scaled = final_scaler.transform(X_test)
y_test_pred = np.argmax(final_model.predict(X_test_scaled), axis=1)

submission_df = pd.DataFrame({"ID": range(len(y_test_pred)), "Predicted": y_test_pred})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_ExpDecay_F1_{mean_f1:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

===== FOLD 1 =====


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.97737
Yeni en iyi F1 => 0.97737, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.5164 - loss: 2.5701 - val_accuracy: 0.9774 - val_loss: 1.2556 - val_f1: 0.9774 - learning_rate: 9.8726e-04
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98014
Yeni en iyi F1 => 0.98014, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5851 - loss: 1.9517 - val_accuracy: 0.9801 - val_loss: 1.0293 - val_f1: 0.9801 - learning_rate: 9.7468e-04
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 3 => val_f1: 0.98169
Yeni en iyi F1 => 0.98169, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6065 - loss: 1.6389 - val_accuracy: 0.9818 - val_loss: 0.8217 - val_f1: 0.9817 - learning_rate: 9.6226e-04
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 4 => val_f1: 0.98298
Yeni en iyi F1 => 0.98298, model kaydedildi.
250/250 ━━━━━━━━

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from google.colab import drive

# 1) Google Drive
drive.mount('/content/drive')

# 2) Dosya Yolları
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# 3) Verileri Yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# Mixup (opsiyonel)
def mixup_data(X, y, alpha=0.2):
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X + (1 - lam) * X_shuffled
    return X_mix, y, lam, y_shuffled

# F1 Callback
class F1Callback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        y_pred_proba = self.model.predict(self.X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(self.y_val, y_pred, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")

# ExponentialDecay ile LR
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=800,
    decay_rate=0.95,
    staircase=False
)

# Model Oluşturma
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.0001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))

    # Bu satırda schedule verilince LR manuel ayarlanamaz
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Ensemble Yapısı
SEED_LIST = [42, 2023, 777]
FOLDS = 5
oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, 10))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, 10))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n========================== SEED = {seed_val} ==========================")
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold_index+1} --- (Seed {seed_val})")
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)

        X_train_mix, y_train_mix, _, _ = mixup_data(X_train_cv, y_train_cv, alpha=0.2)

        model = create_model(X_train_cv.shape[1])

        # Burada ReduceLROnPlateau yok
        f1_cb = F1Callback(X_val_cv, y_val_cv)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv),
            epochs=40,
            batch_size=128,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        model.load_weights("best_f1_model.weights.h5")

        # OOF
        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba

        # Test
        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1

# Stacking (LogisticRegression)
oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

meta_model = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000
)
meta_model.fit(oof_reshaped, y)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y, oof_pred_meta, average='macro')
print("\nFinal Ensemble OOF F1 =>", f1_ensemble)

test_pred_meta = meta_model.predict(test_reshaped)
submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_ensemble_stacking_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

========================== SEED = 42 ==========================

--- Fold 1 --- (Seed 42)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98650
Yeni en iyi F1 => 0.98650, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.9408 - loss: 0.4481 - val_accuracy: 0.9865 - val_loss: 0.2991 - val_f1: 0.9865
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 2 => val_f1: 0.98413
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9886 - loss: 0.2811 - val_accuracy: 0.9841 - val_loss: 0.2828 - val_f1: 0.9841
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 3 => val_f1: 0.98302
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9931 - loss: 0.2387 - val_accuracy: 0.9830 - val_loss: 0.2654 - val_f1: 0.9830
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 4 => val_f1: 0.98032
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9956 - loss: 0.2067 - val_accuracy: 0.9804 - val_loss: 0.2568 - val_f1: 0.9803
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 5 => val_f1: 0.97782
250/250 ━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98463
Yeni en iyi F1 => 0.98463, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8247 - loss: 0.8094 - val_accuracy: 0.9846 - val_loss: 0.3198 - val_f1: 0.9846
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 2 => val_f1: 0.98562
Yeni en iyi F1 => 0.98562, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9366 - loss: 0.4442 - val_accuracy: 0.9856 - val_loss: 0.3107 - val_f1: 0.9856
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 3 => val_f1: 0.98361
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9586 - loss: 0.3788 - val_accuracy: 0.9836 - val_loss: 0.3132 - val_f1: 0.9836
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98276
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9662 - loss: 0.3448 - val_accuracy: 0.9827 - val_loss: 0.3128 - val_f1: 0.9828
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch

...

Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98462
Yeni en iyi F1 => 0.98462, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9411 - loss: 0.4477 - val_accuracy: 0.9846 - val_loss: 0.3053 - val_f1: 0.9846
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 2 => val_f1: 0.98513
Yeni en iyi F1 => 0.98513, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9903 - loss: 0.2756 - val_accuracy: 0.9851 - val_loss: 0.2749 - val_f1: 0.9851
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 3 => val_f1: 0.98525
Yeni en iyi F1 => 0.98525, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9937 - loss: 0.2304 - val_accuracy: 0.9852 - val_loss: 0.2552 - val_f1: 0.9852
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98599
Yeni en iyi F1 => 0.98599, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9936 - loss: 0.2045 - val_accuracy: 0.9860 -

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98287
Yeni en iyi F1 => 0.98287, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9285 - loss: 0.4979 - val_accuracy: 0.9829 - val_loss: 0.3188 - val_f1: 0.9829
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98301
Yeni en iyi F1 => 0.98301, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9874 - loss: 0.2915 - val_accuracy: 0.9830 - val_loss: 0.3006 - val_f1: 0.9830
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98414
Yeni en iyi F1 => 0.98414, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9924 - loss: 0.2493 - val_accuracy: 0.9841 - val_loss: 0.2751 - val_f1: 0.9841
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 4 => val_f1: 0.98061
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9927 - loss: 0.2221 - val_accuracy: 0.9806 - val_loss: 0.2880 - val_f1: 0.9806
Epoch 5/40
2

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98348
Yeni en iyi F1 => 0.98348, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9414 - loss: 0.4512 - val_accuracy: 0.9835 - val_loss: 0.3046 - val_f1: 0.9835
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98486
Yeni en iyi F1 => 0.98486, model kaydedildi.
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9923 - loss: 0.2695 - val_accuracy: 0.9849 - val_loss: 0.2831 - val_f1: 0.9849
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98287
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9947 - loss: 0.2307 - val_accuracy: 0.9829 - val_loss: 0.2643 - val_f1: 0.9829
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98226
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9950 - loss: 0.2001 - val_accuracy: 0.9822 - val_loss: 0.2525 - val_f1: 0.9823
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch

In [ ]:
!pip install lightgbm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from google.colab import drive

# LightGBM Meta-Learner
from lightgbm import LGBMClassifier

# 1) Google Drive
drive.mount('/content/drive')

# 2) Dosya Yolları
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# 3) Verileri Yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y_raw = train_labels['label'].values  # Ham (integer) etiketler

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

num_classes = 10  # 10 sınıf olduğunu varsaydık

# 4) Soft Label Mixup Fonksiyonu
def mixup_data(X_in, y_in_onehot, alpha=0.2):
    indices = np.random.permutation(len(X_in))
    X_shuffled = X_in[indices]
    y_shuffled_onehot = y_in_onehot[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X_in + (1 - lam) * X_shuffled
    y_mix = lam * y_in_onehot + (1 - lam) * y_shuffled_onehot  # soft label
    return X_mix, y_mix

# 5) F1 Callback - Soft Label Versiyon
class F1Callback(Callback):
    def __init__(self, X_val, y_val_onehot):
        super().__init__()
        self.X_val = X_val
        self.y_val_onehot = y_val_onehot  # One-hot validation etiketleri
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        # Tahmin (softmax)
        y_pred_proba = self.model.predict(self.X_val)
        # y_val_onehot => shape [val_size, num_classes]
        # Convert them to integer classes
        true_classes = np.argmax(self.y_val_onehot, axis=1)
        pred_classes = np.argmax(y_pred_proba, axis=1)

        f1_val = f1_score(true_classes, pred_classes, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")

# 6) Exponential Decay - LR
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-4,  # Daha yavaş başlangıç LR
    decay_steps=1000,
    decay_rate=0.95,
    staircase=False
)

# 7) Model Oluşturma (düşük dropout, ufak L2)
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.00005), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.15))

    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.15))

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.15))

    model.add(Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    # Soft label => 'categorical_crossentropy'
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 8) Büyük Ensemble (5 Seeds x 5 Folds = 25 Model)
SEED_LIST = [42, 2023, 777, 999, 314159]
FOLDS = 5

# OOF ve Test tahminlerini saklayacak
oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, num_classes))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, num_classes))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n=================== SEED = {seed_val} ===================")
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y_raw)):
        print(f"\n--- Fold {fold_index+1} / Seed {seed_val} ---")

        # Ayrıştırma
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y_raw[train_idx], y_raw[val_idx]

        # One-hot
        y_train_cv_onehot = tf.keras.utils.to_categorical(y_train_cv, num_classes=num_classes)
        y_val_cv_onehot = tf.keras.utils.to_categorical(y_val_cv, num_classes=num_classes)

        # Ölçekleme
        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)

        # Mixup (soft labels)
        X_train_mix, y_train_mix = mixup_data(X_train_cv, y_train_cv_onehot, alpha=0.2)

        # Model
        model = create_model(X_train_cv.shape[1])

        # Callback
        f1_cb = F1Callback(X_val_cv, y_val_cv_onehot)
        early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

        # Eğit
        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv_onehot),
            epochs=50,
            batch_size=64,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        # En iyi modelin weights'ini yükle
        model.load_weights("best_f1_model.weights.h5")

        # OOF
        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba

        # Test
        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1

# 9) Meta-Özellikleri Hazırla
oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

# 10) LightGBM Meta-Learner
meta_model = LGBMClassifier(
    n_estimators=800,
    learning_rate=0.02,
    num_leaves=31,
    max_depth=-1,
    random_state=42
)

# OOF verisi -> meta model
meta_model.fit(oof_reshaped, y_raw)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y_raw, oof_pred_meta, average='macro')
print(f"\nFinal Ensemble OOF F1 => {f1_ensemble:.5f}")

# Test son tahmin
test_pred_meta = meta_model.predict(test_reshaped)
submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_big_ensemble_lgbm_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=================== SEED = 42 ===================

--- Fold 1 / Seed 42 ---


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98626
Yeni en iyi F1 => 0.98626, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9400 - loss: 0.4243 - val_accuracy: 0.9862 - val_loss: 0.1756 - val_f1: 0.9863
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98637
Yeni en iyi F1 => 0.98637, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9911 - loss: 0.2622 - val_accuracy: 0.9864 - val_loss: 0.1719 - val_f1: 0.9864
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 3 => val_f1: 0.98813
Yeni en iyi F1 => 0.98813, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9982 - loss: 0.2357 - val_accuracy: 0.9881 - val_loss: 0.1625 - val_f1: 0.9881
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 4 => val_f1: 0.98936
Yeni en iyi F1 => 0.98936, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9994 - loss: 0.2190 - val_accuracy: 0.9894 - 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63712
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 250
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Final Ensemble OOF F1 => 1.00000


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Submission kaydedildi => /content/drive/My Drive/Colab_Data/submission_big_ensemble_lgbm_F1_1.00000.csv


In [ ]:
pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
!pip install lightgbm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AlphaDropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from google.colab import drive

# LightGBM meta-learner
from lightgbm import LGBMClassifier

# ------------------------------------------------------
# 1) Google Drive
# ------------------------------------------------------
drive.mount('/content/drive')

# ------------------------------------------------------
# 2) Dosya Yolları
# ------------------------------------------------------
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# ------------------------------------------------------
# 3) Verileri Yükle
# ------------------------------------------------------
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y_raw = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

num_classes = 10  # 10 sınıf olduğunu varsayıyoruz

# ------------------------------------------------------
# 4) Soft Label Mixup (alpha=0.4)
# ------------------------------------------------------
def mixup_data(X_in, y_in_onehot, alpha=0.4):
    indices = np.random.permutation(len(X_in))
    X_shuffled = X_in[indices]
    y_shuffled_onehot = y_in_onehot[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X_in + (1 - lam) * X_shuffled
    y_mix = lam * y_in_onehot + (1 - lam) * y_shuffled_onehot  # soft label
    return X_mix, y_mix

# ------------------------------------------------------
# 5) F1 Callback (Soft Label Versiyon)
# ------------------------------------------------------
class F1Callback(Callback):
    def __init__(self, X_val, y_val_onehot):
        super().__init__()
        self.X_val = X_val
        self.y_val_onehot = y_val_onehot
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        y_pred_proba = self.model.predict(self.X_val)
        true_classes = np.argmax(self.y_val_onehot, axis=1)
        pred_classes = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(true_classes, pred_classes, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")

# ------------------------------------------------------
# 6) ExponentialDecay LR
# ------------------------------------------------------
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-4,
    decay_steps=1000,
    decay_rate=0.95,
    staircase=False
)

# ------------------------------------------------------
# 7) SELU + AlphaDropout Model
# ------------------------------------------------------
def create_model(input_dim):
    model = Sequential()
    # 1. Katman
    model.add(Dense(1024, activation='selu', kernel_regularizer=l2(0.00005), input_shape=(input_dim,)))
    model.add(AlphaDropout(0.15))

    # 2. Katman
    model.add(Dense(512, activation='selu', kernel_regularizer=l2(0.00005)))
    model.add(AlphaDropout(0.15))

    # 3. Katman
    model.add(Dense(256, activation='selu', kernel_regularizer=l2(0.00005)))
    model.add(AlphaDropout(0.15))

    # Çıkış
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    # Soft label => categorical_crossentropy
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ------------------------------------------------------
# 8) Ensemble (7 Seeds x 5 Folds = 35 model)
# ------------------------------------------------------
SEED_LIST = [42, 2023, 777, 999, 314159, 2024, 1234]  # 7 seed
FOLDS = 5

oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, num_classes))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, num_classes))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n================== SEED = {seed_val} ==================")
    # tf.random.set_seed(seed_val)  # KALDIRILDI
    np.random.seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y_raw)):
        print(f"\n--- Fold {fold_index+1} / Seed {seed_val} ---")

        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y_raw[train_idx], y_raw[val_idx]

        # One-hot
        y_train_cv_onehot = tf.keras.utils.to_categorical(y_train_cv, num_classes=num_classes)
        y_val_cv_onehot = tf.keras.utils.to_categorical(y_val_cv, num_classes=num_classes)

        # Ölçekleme
        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)

        # Mixup
        X_train_mix, y_train_mix = mixup_data(X_train_cv, y_train_cv_onehot, alpha=0.4)

        # Model
        model = create_model(X_train_cv.shape[1])

        # Callback
        f1_cb = F1Callback(X_val_cv, y_val_cv_onehot)
        early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv_onehot),
            epochs=50,
            batch_size=64,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        # En iyi ağırlıkları yükle
        model.load_weights("best_f1_model.weights.h5")

        # OOF
        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba

        # Test
        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1

# ------------------------------------------------------
# 9) Meta-Özellikleri Hazırla
# ------------------------------------------------------
oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

# ------------------------------------------------------
# 10) LightGBM Meta-Learner
# ------------------------------------------------------
meta_model = LGBMClassifier(
    n_estimators=1500,
    learning_rate=0.01,
    num_leaves=63,
    max_depth=-1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    random_state=42
)

meta_model.fit(oof_reshaped, y_raw)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y_raw, oof_pred_meta, average='macro')
print(f"\nFinal Ensemble OOF F1 => {f1_ensemble:.5f}")

# ------------------------------------------------------
# 11) Test Set Tahmini ve Submission
# ------------------------------------------------------
test_pred_meta = meta_model.predict(test_reshaped)

submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_big_ensemble_lgbm_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

================== SEED = 42 ==================

--- Fold 1 / Seed 42 ---


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


TypeError: Exception encountered when calling AlphaDropout.call().

[1mgreater_equal() got an unexpected keyword argument 'seed'[0m

Arguments received by AlphaDropout.call():
  • inputs=tf.Tensor(shape=(64, 1024), dtype=float32)
  • training=True

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from google.colab import drive

# 1) Google Drive
drive.mount('/content/drive')

# 2) Dosya Yolları
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# 3) Verileri Yükle
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# 4) Mixup (hard label)
def mixup_data(X, y, alpha=0.3):
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X + (1 - lam) * X_shuffled
    # y değişmiyor, hard label
    return X_mix, y, lam, y_shuffled

# 5) F1 Callback
class F1Callback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        y_pred_proba = self.model.predict(self.X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(self.y_val, y_pred, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")

# 6) ExponentialDecay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=800,
    decay_rate=0.95,
    staircase=False
)

# 7) Model Oluşturma (4 katman, l2=0.00005)
def create_model(input_dim):
    model = Sequential()

    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.00005), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 8) Ensemble Yapısı
SEED_LIST = [42, 2023, 777, 999, 314159, 2024, 1234]  # 7 seeds
FOLDS = 5

oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, 10))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, 10))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n========================== SEED = {seed_val} ==========================")
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold_index+1} --- (Seed {seed_val})")

        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        # Ölçekleme
        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)

        # Mixup
        X_train_mix, y_train_mix, _, _ = mixup_data(X_train_cv, y_train_cv, alpha=0.3)

        model = create_model(X_train_cv.shape[1])

        f1_cb = F1Callback(X_val_cv, y_val_cv)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv),
            epochs=50,
            batch_size=64,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        model.load_weights("best_f1_model.weights.h5")

        # OOF
        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba

        # Test
        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1

# 9) Stacking (LogisticRegression) - C=3.0
oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

meta_model = LogisticRegression(
    penalty='l2',
    C=3.0,  # Biraz daha esnek
    solver='lbfgs',
    max_iter=2000
)
meta_model.fit(oof_reshaped, y)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y, oof_pred_meta, average='macro')
print("\nFinal Ensemble OOF F1 =>", f1_ensemble)

test_pred_meta = meta_model.predict(test_reshaped)
submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_ensemble_stacking_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

========================== SEED = 42 ==========================

--- Fold 1 --- (Seed 42)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98314
Yeni en iyi F1 => 0.98314, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9304 - loss: 0.3737 - val_accuracy: 0.9831 - val_loss: 0.1999 - val_f1: 0.9831
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98298
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9852 - loss: 0.1934 - val_accuracy: 0.9830 - val_loss: 0.2088 - val_f1: 0.9830
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 3 => val_f1: 0.98300
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9903 - loss: 0.1706 - val_accuracy: 0.9830 - val_loss: 0.1946 - val_f1: 0.9830
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.98402
Yeni en iyi F1 => 0.98402, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9937 - loss: 0.1519 - val_accuracy: 0.9840 - val_loss: 0.1936 - val_f1: 0.9840
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.97689
Yeni en iyi F1 => 0.97689, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.7240 - loss: 0.9456 - val_accuracy: 0.9769 - val_loss: 0.2160 - val_f1: 0.9769
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 2 => val_f1: 0.97929
Yeni en iyi F1 => 0.97929, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8526 - loss: 0.5427 - val_accuracy: 0.9793 - val_loss: 0.2230 - val_f1: 0.9793
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98002
Yeni en iyi F1 => 0.98002, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8933 - loss: 0.4498 - val_accuracy: 0.9800 - val_loss: 0.2330 - val_f1: 0.9800
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98026
Yeni en iyi F1 => 0.98026, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9162 - loss: 0.3974 - val_accuracy: 0.9803 - 

...

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98200
Yeni en iyi F1 => 0.98200, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8747 - loss: 0.5480 - val_accuracy: 0.9820 - val_loss: 0.2097 - val_f1: 0.9820
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98388
Yeni en iyi F1 => 0.98388, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9606 - loss: 0.2679 - val_accuracy: 0.9839 - val_loss: 0.2118 - val_f1: 0.9839
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98115
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9774 - loss: 0.2193 - val_accuracy: 0.9811 - val_loss: 0.2193 - val_f1: 0.9811
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98203
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9808 - loss: 0.2118 - val_accuracy: 0.9820 - val_loss: 0.2315 - val_f1: 0.9820
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.25316
Yeni en iyi F1 => 0.25316, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.1408 - loss: 2.7183 - val_accuracy: 0.2860 - val_loss: 1.8621 - val_f1: 0.2532
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.31098
Yeni en iyi F1 => 0.31098, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3986 - loss: 1.8176 - val_accuracy: 0.3115 - val_loss: 1.7085 - val_f1: 0.3110
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.31159
Yeni en iyi F1 => 0.31159, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5551 - loss: 1.3864 - val_accuracy: 0.3234 - val_loss: 1.7323 - val_f1: 0.3116
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.41554
Yeni en iyi F1 => 0.41554, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6447 - loss: 1.1644 - val_accuracy: 0.4149 - 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98250
Yeni en iyi F1 => 0.98250, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9355 - loss: 0.3539 - val_accuracy: 0.9825 - val_loss: 0.2009 - val_f1: 0.9825
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98499
Yeni en iyi F1 => 0.98499, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9865 - loss: 0.1874 - val_accuracy: 0.9850 - val_loss: 0.1914 - val_f1: 0.9850
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98463
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9913 - loss: 0.1641 - val_accuracy: 0.9846 - val_loss: 0.1893 - val_f1: 0.9846
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.98177
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9942 - loss: 0.1485 - val_accuracy: 0.9818 - val_loss: 0.2005 - val_f1: 0.9818
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoc

In [ ]:
!pip install lightgbm

In [ ]:
!pip install lightgbm

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from google.colab import drive

# LightGBM meta-learner
from lightgbm import LGBMClassifier

# ------------------------------------------------------
# 1) Google Drive
# ------------------------------------------------------
drive.mount('/content/drive')

# ------------------------------------------------------
# 2) Dosya Yolları
# ------------------------------------------------------
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# ------------------------------------------------------
# 3) Verileri Yükle
# ------------------------------------------------------
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

# ------------------------------------------------------
# 4) Mixup
# ------------------------------------------------------
def mixup_data(X_in, y_in, alpha=0.3):
    indices = np.random.permutation(len(X_in))
    X_shuffled = X_in[indices]
    y_shuffled = y_in[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X_in + (1 - lam) * X_shuffled
    # Hard label: y karıştırmıyoruz
    return X_mix, y_in, lam, y_shuffled

# ------------------------------------------------------
# 5) F1 Callback
# ------------------------------------------------------
class F1Callback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        y_pred_proba = self.model.predict(self.X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(self.y_val, y_pred, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")

# ------------------------------------------------------
# 6) ExponentialDecay LR
# ------------------------------------------------------
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=800,
    decay_rate=0.95,
    staircase=False
)

# ------------------------------------------------------
# 7) Model Oluşturma (4 katman, l2=0.00005)
# ------------------------------------------------------
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.00005), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ------------------------------------------------------
# 8) Ensemble (10 Seeds x 5 Folds = 50 model)
# ------------------------------------------------------
SEED_LIST = [42, 2023, 777, 999, 314159, 2024, 1234, 1111, 5555, 9876]
FOLDS = 5

oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, 10))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, 10))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n========================== SEED = {seed_val} ==========================")
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold_index+1} --- (Seed {seed_val})")

        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        # Ölçekleme
        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)

        # Mixup
        X_train_mix, y_train_mix, _, _ = mixup_data(X_train_cv, y_train_cv, alpha=0.3)

        model = create_model(X_train_cv.shape[1])

        f1_cb = F1Callback(X_val_cv, y_val_cv)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv),
            epochs=50,
            batch_size=64,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        # En iyi ağırlıkları geri yükle
        model.load_weights("best_f1_model.weights.h5")

        # OOF
        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba

        # Test
        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1

# ------------------------------------------------------
# 9) Meta-Özellikleri Hazırla
# ------------------------------------------------------
oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

# ------------------------------------------------------
# 10) LightGBM Meta-Learner
# ------------------------------------------------------
meta_model = LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.01,
    num_leaves=63,
    max_depth=-1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    random_state=42
)

meta_model.fit(oof_reshaped, y)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y, oof_pred_meta, average='macro')
print("\nFinal Ensemble OOF F1 =>", f1_ensemble)

# ------------------------------------------------------
# 11) Test Set Tahmini ve Submission
# ------------------------------------------------------
test_pred_meta = meta_model.predict(test_reshaped)
submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_big_ensemble_lgbm_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Mounted at /content/drive

========================== SEED = 42 ==========================

--- Fold 1 --- (Seed 42)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
Epoch 1 => val_f1: 0.98359
Yeni en iyi F1 => 0.98359, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 49s 91ms/step - accuracy: 0.9331 - loss: 0.3671 - val_accuracy: 0.9836 - val_loss: 0.2013 - val_f1: 0.9836
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step
Epoch 2 => val_f1: 0.98613
Yeni en iyi F1 => 0.98613, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 45s 91ms/step - accuracy: 0.9853 - loss: 0.1911 - val_accuracy: 0.9861 - val_loss: 0.1924 - val_f1: 0.9861
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 3 => val_f1: 0.98347
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 0.9913 - loss: 0.1642 - val_accuracy: 0.9835 - val_loss: 0.1975 - val_f1: 0.9835
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 4 => val_f1: 0.98429
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 90ms/step - accuracy: 0.9928 - loss: 0.1537 - val_accuracy: 0.9843 - val_loss: 0.1960 - val_f1: 0.9843
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/s

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 1 => val_f1: 0.97989
Yeni en iyi F1 => 0.97989, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 95ms/step - accuracy: 0.7216 - loss: 0.9520 - val_accuracy: 0.9799 - val_loss: 0.2122 - val_f1: 0.9799
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Epoch 2 => val_f1: 0.97988
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 87ms/step - accuracy: 0.8509 - loss: 0.5448 - val_accuracy: 0.9799 - val_loss: 0.2206 - val_f1: 0.9799
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 3 => val_f1: 0.98313
Yeni en iyi F1 => 0.98313, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.8922 - loss: 0.4458 - val_accuracy: 0.9831 - val_loss: 0.2254 - val_f1: 0.9831
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 4 => val_f1: 0.98351
Yeni en iyi F1 => 0.98351, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 90ms/step - accuracy: 0.9146 - loss: 0.4021 - val_accuracy: 0.9835 - val_loss: 0.2398 - val_f1: 0.9835
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Epoch 1 => val_f1: 0.98021
Yeni en iyi F1 => 0.98021, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 88ms/step - accuracy: 0.9328 - loss: 0.3645 - val_accuracy: 0.9803 - val_loss: 0.2126 - val_f1: 0.9802
Epoch 2/50
369/500 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.9856 - loss: 0.1911 